In [2]:
# 딥러닝 기본 모델 사용할 때 참고해본다.

import math

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model

In [3]:
BATCH_SIZE = 128
SAVED_MODEL_DIR = './saved_model'

In [4]:
(ds_train_data, ds_val_data), info = tfds.load(
    name='mnist',
    split=['train', 'test'],
    with_info=True,
    as_supervised=True,
)

num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-train.tfrecord...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to C:\Users\lemon\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [5]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = (
    ds_train_data
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(info.splits['train'].num_examples)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

ds_val = (
    ds_val_data
    .map(preprocess, AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTOTUNE)
)

In [6]:
inputs = layers.Input(shape=(28, 28, 1), name='input')

x = layers.Conv2D(24, kernel_size=(6, 6), strides=1)(inputs)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

x = layers.Conv2D(48, kernel_size=(5, 5), strides=2)(x)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

x = layers.Conv2D(64, kernel_size=(4, 4), strides=2)(x)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

x = layers.Flatten()(x)
x = layers.Dense(200)(x)
x = layers.BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = layers.Activation('relu')(x)
x = layers.Dropout(rate=0.25)(x)

predications = layers.Dense(num_classes, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=predications)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 23, 23, 24)        888       
_________________________________________________________________
batch_normalization (BatchNo (None, 23, 23, 24)        72        
_________________________________________________________________
activation (Activation)      (None, 23, 23, 24)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 48)        28848     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 48)        144   

In [8]:
lr_decay = lambda epoch: 0.0001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
    callbacks=[decay_callback],
    verbose=1
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0201.
Epoch 1/20
469/469 [==============================] - 8s 17ms/step - loss: 0.1591 - accuracy: 0.9503 - val_loss: 0.0749 - val_accuracy: 0.9787

Epoch 00002: LearningRateScheduler reducing learning rate to 0.014430626211475785.
Epoch 2/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0606 - accuracy: 0.9819 - val_loss: 0.0313 - val_accuracy: 0.9906

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01036834238065184.
Epoch 3/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0447 - accuracy: 0.9860 - val_loss: 0.0286 - val_accuracy: 0.9904

Epoch 00004: LearningRateScheduler reducing learning rate to 0.007457588823428847.
Epoch 4/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0348 - accuracy: 0.9893 - val_loss: 0.0201 - val_accuracy: 0.9934

Epoch 00005: LearningRateScheduler reducing learning rate to 0.005371942762314537.
Epoch 5/20
469/469 [======

In [10]:
tf.saved_model.save(model, SAVED_MODEL_DIR)

INFO:tensorflow:Assets written to: ./saved_model\assets


INFO:tensorflow:Assets written to: ./saved_model\assets


In [11]:
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

with open('mnist.tflite', 'wb') as f:
    f.write(tflite_model)

In [12]:
try:
    from google.colab import files
    files.download('mnist.tflite')
except:
    print("Skip downloading")

Skip downloading
